In [ ]:

def extract_legal_proceedings(text):
    """ 2. The content of “Item 3. LEGAL PROCEEDINGS”."""
    # TODO: Not sure which implementation is correct
    legal_proceedings = re.search(r"Item 3. LEGAL PROCEEDINGS(.*?)Item 4", text, re.IGNORECASE|re.DOTALL).group(1)

    # legal_proceedings = re.search(r"Item 3\. Legal Proceedings(.*?)(Item 4\.|$)", text, re.DOTALL).group(1).strip()
    return legal_proceedings

def extract_signature_date(text):
    """ 3. The date of signature(s) (ensure it is formatted in ISO-format)."""
    # TODO: Not sure which implementation is correct
    # signature_date = re.search(r"SIGNATURES(.*?)</html>", text, re.IGNORECASE|re.DOTALL).group(1)

    signature_date = re.search(r"Date:\s*(\d{4}-\d{2}-\d{2})", text).group(1)

    return signature_date

def extract_signatures(text):
    """ 4. Who signed the report? If there are multiple signatures, all of them have to be listed (comma separated).
    Do not include the audit firm – in case it is given in the report."""
    # TODO: Not sure which implementation is correct
    signers = re.findall(r"(?:By|Signed by):\s*([A-Za-z ]+)\n", text)
    # signers = ", ".join([s.strip() for s in signers if "audit" not in s.lower()])

    signatures = re.findall(r"SIGNATURES(.*?)</html>", text, re.IGNORECASE|re.DOTALL)
    return signatures

In [ ]:
"""
Step 4: Process the whole file
"""
def process_file(file):
    """ This function processes a single 10-k file and extracts the necessary information """
    clean_content = clean_html(file)

    fiscal_year = extract_fiscal_year(clean_content)

    legal_proceedings = extract_legal_proceedings(clean_content)

    signature_date = extract_signature_date(clean_content)

    signers = extract_signatures(clean_content)
    return {
        "fiscal_year": fiscal_year,
        "legal_proceedings": legal_proceedings,
        "signature_date": signature_date,
        "signers": signers
    }

In [2]:
"""
Step 1: Import necessary libraries
I have moved the iterations to the very end
"""
import re
from pathlib import Path
import pandas as pd
from datetime import datetime

"""
Step 2: Prepare a function to clean the raw content, removing html tags and entities
"""
def clean_html(raw_html):
    clean_one = re.sub(r'<.*?>', ' ', raw_html)
    clean_two = re.sub(r'&\w+;', ' ', clean_one)
    clean_three = re.sub(r"&[a-z]+;", " ", clean_two)
    clean_four = re.sub(r'\s+', ' ', clean_three)
    return clean_four.strip()

"""
Step 3: Extract all necessary information from the cleaned text
"""
def extract_fiscal_year(text):
    """ 1. The date of the fiscal year-end (ensure it is formatted in ISO-format)

    Currently, relevant information is in the beginning of the document, e.g.:
     'For the fiscal year ended January 31, 2024, or'
     Search for the fiscal year and return it in ISO-format.
     """
    match = re.search(r"fiscal year ended (\w+\s\d{1,2},\s\d{4})", text, re.IGNORECASE)
    if match:
        fiscal_year_str = match.group(1)
        fiscal_year_date = datetime.strptime(fiscal_year_str, "%B %d, %Y").date()
        fiscal_year_iso = fiscal_year_date.isoformat()
    else:
        fiscal_year_iso = 'N/A'
    return fiscal_year_iso

def extract_legal_proceedings(text):
    """ 2. The content of “Item 3. LEGAL PROCEEDINGS”."""

    match = re.search(r"Item 3\. Legal Proceedings(.*?)(Item 4\.|$)", text, re.IGNORECASE|re.DOTALL)
    return match.group(1).strip() if match else 'N/A'

def extract_signature_date(text):
    """ 3. The date of signature(s) (ensure it is formatted in ISO-format).

    Find the SIGNATURES section

    """
    match = re.search(r"Date:\s*(\d{4}-\d{2}-\d{2})", text)
    return match.group(1) if match else 'N/A'

def extract_signatures(text):
    signers = re.findall(r"(?:By|Signed by):\s*([A-Za-z ]+)\n", text)
    signers = ", ".join([s.strip() for s in signers if "audit" not in s.lower()])
    return signers if signers else 'N/A'

def process_file(file):
    clean_content = clean_html(file)
    fiscal_year = extract_fiscal_year(clean_content)
    legal_proceedings = extract_legal_proceedings(clean_content)
    signature_date = extract_signature_date(clean_content)
    signers = extract_signatures(clean_content)
    return {
        "fiscal_year": fiscal_year,
        "legal_proceedings": legal_proceedings,
        "signature_date": signature_date,
        "signers": signers
    }

current_directory = Path.cwd()
results = []
for html_file in current_directory.glob('*.html'):
    with html_file.open('r', encoding='utf-8') as file:
        raw_content = file.read()
        result = process_file(raw_content)
        if result:
            results.append(result)

df = pd.DataFrame(results)
df.to_csv('10k_results.csv', index=False)
df.head(10)

,fiscal_year,legal_proceedings,signature_date,signers
0,2024-01-31,","" under the sub-caption ""I. Supplemental Info...",N/A,N/A
1,N/A,25,N/A,N/A
2,2016-01-03,The information called for by this item is inc...,N/A,N/A
3,N/A,As a multinational company with a complex and ...,N/A,N/A
4,N/A,18,N/A,N/A
5,N/A,&#8221; of this report. Increased or new indir...,N/A,N/A
6,2024-06-30,10,N/A,N/A
7,2024-06-30,Refer to Note 15 &#8211; Contingencies of the ...,N/A,N/A
8,N/A,18,N/A,N/A
9,2023-12-31,30,N/A,N/A
